All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 서적
    - Python Machine Learning(3rd Ed.), Sebastian Raschka , Vahid Mirjalili, Packt, 2019.10.
        - 8장. Applying Machine Learning to Sentiment Analysis 코드 참조.
    - 텐서플로2와 머신러닝으로 시작하는 자연어 처리, 전창욱, 최태균, 조중현, 신성진 지음, 위키북스, 2020.
        - 4장. 텍스트 분류 예제 참조.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
DATA_IN_PATH = './data_in/' 
TRAIN_CLEAN_DATA = 'train_clean.csv'
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [ ]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
#min_df: ignore terms that have df strictly lower than the given threshold.
#analyzer=[word, char, char_wb] : feature should be made of word or character n-grams
#sublinear_tf: Apply sublinear tf scaling, i.e. replace tf with 1+log(tf).
#ngram_range: lower and upper boundary of the range of n-values for n-grams 
#max_features: build a vocabulary that only consider the top max_features ordered by TF 
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", 
            sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [ ]:
print(X.shape)
#print(y.shape)
#X1 = vectorizer.fit(reviews)
#print(X1.vocabulary_)

In [ ]:
idf = vectorizer.idf_
features = vectorizer.get_feature_names()
#print(dict(zip(vectorizer.get_feature_names(), idf)))

In [ ]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2
X_train, X_eval, y_train, y_eval = train_test_split(X, y, 
                test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [ ]:
lgs = LogisticRegression(class_weight='balanced') 
lgs.fit(X_train, y_train) 

In [ ]:
predicted = lgs.predict(X_eval)

In [ ]:
print(predicted[277])
print(y_eval[277])
#print(predicted[1000])
#print(y_eval[1000])

In [ ]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

In [ ]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [ ]:
testDataVecs = vectorizer.transform(test_data['review'])

In [ ]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

In [ ]:
import csv
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/' 
test_file = DATA_IN_PATH + 'ratings_test.txt'
test_data = pd.read_csv(test_file, header=0, delimiter='\t', quoting=csv.QUOTE_NONE)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)